In [9]:
import numpy as np
import time
import numba
from numba import prange

from sklearn.neighbors import BallTree
import pandas as pd

In [2]:
drop_columns = ['DNA1', 'AF488', 'AF555', 'AF647', 'DNA2', 'BG488', 'BG555', 'BG647', 'DNA3', 'CD2', 'NONE', 'CD10', 'DNA4', 'CD30', 'ALK', 'CD5', 'DNA5', 'CD4', 'CD68', 'CD7', 'DNA6', 'CD43', 'CD3D', 'CD45', 'DNA7', 'CD11B', 'GATA3', 'CD8A', 'DNA8', 'CD163', 'CD19', 'CD56', 'DNA10', 'KI67', 'FOXP3', 'CD21', 'DNA11', 'S6P', 'ERKP', 'CD31', 'DNA12', 'BCL6', 'CD57', 'PSTAT3']

In [3]:
df = pd.read_csv('/Users/swarchol/Harvard Drive/Ton/tonsil_with_phenotype.csv', index_col=None)
df = df.drop(drop_columns, axis=1)
df['id'] = df.index
# df['Cluster'] = embedding[:, -1].astype('int32').tolist()
df['Cluster'] = 0

if 'CellType' in df.columns:
    df = df.rename(columns={'CellType': 'phenotype'})
df = df.replace(-np.Inf, 0)
np_df = df.to_numpy()

In [4]:
test = time.time()
points = np_df[:, [1,2]].astype(int)
ton_scale_factor = 0.649999976158
r = 100 / ton_scale_factor



In [5]:
contiguous_points = np.ascontiguousarray(points.astype('float32'))
image_ball_tree = BallTree(contiguous_points, metric='euclidean')
neighbors, distances = image_ball_tree.query_radius(contiguous_points, r=r, return_distance=True)


In [11]:
@numba.jit(nopython=True, parallel=True)
def create_matrices(neighbors, distances):
    lengths = np.zeros((len(neighbors),))
    max_neighbors = 0
    for i in prange(len(neighbors)):
        lengths[i] = len(neighbors[i])
        if max_neighbors < len(neighbors[i]):
            max_neighbors = len(neighbors[i])
    neighbors_matrix = np.zeros((len(neighbors), max_neighbors))
    distances_matrix = np.zeros((len(neighbors), max_neighbors))

    for i in prange(len(neighbors)):
        neighbors_matrix[i, 0:len(neighbors[i])] = neighbors[i]
        distances_matrix[i, 0:len(distances[i])] = 1 - (distances[i] / r)

    lengths = lengths.astype('uint16')
    neighbors_matrix = neighbors_matrix.astype('uint32')
    distances_matrix = distances_matrix.astype('ufloat32')
    return lengths,neighbors_matrix, distances_matrix

In [12]:
create_matrices(neighbors,distances)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at /var/folders/ps/x7v31pds3gn4t11djwg0t03h0000gp/T/ipykernel_90463/4013336284.py (3)[0m
[1m
File "../../../../var/folders/ps/x7v31pds3gn4t11djwg0t03h0000gp/T/ipykernel_90463/4013336284.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m


In [30]:
phenotypes_dict = {val: idx for idx, val in enumerate(sorted(df.phenotype.unique()))}
phenotypes_array = np_df[:, 12]
for i in range(phenotypes_array.shape[0]):
    phenotypes_array[i] = phenotypes_dict[phenotypes_array[i]]
phenotypes_array = np.array(phenotypes_array, dtype='uint16').flatten()

In [27]:
np_df[0,12]

'CD8 T cells'

array([ 252, 1059,    0,  433,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [31]:
np.random.choice(neighbors)

array([24669, 25427, 26245, 24765])